In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Configuration
sns.set_style("whitegrid")
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# --- LOAD DATA & MODEL ---
data_path = '../data/processed/'
model_path = '../models/house_price_model.joblib'

print("⏳ Loading data and model...")

# Load Test Features
X_test = pd.read_parquet(f'{data_path}X_test_processed.parquet')

# Load Test Targets (Log Scale)
y_test_log = pd.read_parquet(f'{data_path}y_test_log.parquet')['Price']

# Convert to Real Dollars
y_test_real = np.expm1(y_test_log)

# Load Model
model = joblib.load(model_path)

print(f"✅ Loaded Model: {type(model).__name__}")
print(f"   Test Data Shape: {X_test.shape}")

In [ ]:
# --- GENERATE PREDICTIONS ---
print("🔮 Generating predictions...")

# Predict (Log Scale)
y_pred_log = model.predict(X_test)

# Convert to Real Dollars
y_pred = np.expm1(y_pred_log)

# Create a DataFrame for analysis
results_df = pd.DataFrame({
    'Actual': y_test_real,
    'Predicted': y_pred
})

# Calculate Residuals (Error)
results_df['Error'] = results_df['Actual'] - results_df['Predicted']
results_df['Abs_Error'] = results_df['Error'].abs()

print("✅ Predictions complete.")
results_df.head()

In [ ]:
# --- METRICS ---
mae = mean_absolute_error(y_test_real, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred))
r2 = r2_score(y_test_real, y_pred)

print("🏆 FINAL TEST METRICS 🏆")
print(f"   MAE:  ${mae:,.0f}")
print(f"   RMSE: ${rmse:,.0f}")
print(f"   R2:   {r2:.4f}")

In [ ]:
# --- VISUALIZATION 1: ACTUAL vs PREDICTED ---
plt.figure(figsize=(10, 6))
sns.scatterplot(data=results_df, x='Actual', y='Predicted', alpha=0.5, color='blue', edgecolor=None)

# Perfect prediction line
plt.plot([results_df['Actual'].min(), results_df['Actual'].max()], 
         [results_df['Actual'].min(), results_df['Actual'].max()], 
         'r--', lw=3, label='Perfect Prediction')

plt.title("Actual vs Predicted Prices")
plt.xlabel("Actual Price ($)")
plt.ylabel("Predicted Price ($)")
plt.legend()
plt.show()

In [ ]:
# --- VISUALIZATION 2: RESIDUAL PLOT ---
# Check for patterns in errors (Heteroscedasticity)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=results_df, x='Predicted', y='Error', alpha=0.5, color='purple', edgecolor=None)
plt.axhline(0, color='red', linestyle='--', lw=2)

plt.title("Residual Plot (Predicted vs Error)")
plt.xlabel("Predicted Price ($)")
plt.ylabel("Error (Actual - Predicted)")
plt.show()

In [ ]:
# --- VISUALIZATION 3: ERROR DISTRIBUTION ---
# Are errors normally distributed?
plt.figure(figsize=(10, 6))
sns.histplot(results_df['Error'], bins=50, kde=True, color='orange')
plt.title("Distribution of Prediction Errors")
plt.xlabel("Error ($)")
plt.show()

In [ ]:
# --- VISUALIZATION 4: FEATURE IMPORTANCE ---
# Check if model supports feature importance
if hasattr(model, 'feature_importances_'):
    importances = model.feature_importances_
    features = X_test.columns
    
    feat_df = pd.DataFrame({'Feature': features, 'Importance': importances})
    feat_df = feat_df.sort_values(by='Importance', ascending=False).head(20)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feat_df, y='Feature', x='Importance', palette='magma')
    plt.title("Top 20 Features Driving House Prices")
    plt.show()
else:
    print("Model does not support feature importance.")

In [ ]:
# --- ANALYSIS: WORST PREDICTIONS ---
print("\n📉 TOP 10 WORST PREDICTIONS (High Error)")
worst_predictions = results_df.sort_values(by='Abs_Error', ascending=False).head(10)

# Format for easier reading
display(worst_predictions.style.format({
    'Actual': '${:,.0f}', 
    'Predicted': '${:,.0f}', 
    'Error': '${:,.0f}',
    'Abs_Error': '${:,.0f}'
}))

In [ ]:
# --- DEEP DIVE: DIAGNOSING THE WORST PREDICTIONS ---
# We need to merge the "Worst Predictions" back with their original features (X_test)
# to see WHY they failed (e.g., Are they all Mansions? Do they have missing BuildingArea?)

# 1. Get the indices of the worst predictions
worst_indices = worst_predictions.index

# 2. Fetch their feature values from X_test
# Note: X_test is already processed (scaled/encoded), so it might be hard to read.
# Ideally, we would look at the raw data, but we can still check key indicators.
worst_combined = X_test.loc[worst_indices].copy()

# 3. Add the Prediction info back
worst_combined['Actual_Price'] = worst_predictions['Actual']
worst_combined['Predicted_Price'] = worst_predictions['Predicted']
worst_combined['Error'] = worst_predictions['Error']

# 4. Check for "Luxury Bias"
# If Actual Price is consistently > $3M, our model struggles with luxury homes.
print("💰 Price Analysis of Failures:")
print(worst_combined[['Actual_Price', 'Predicted_Price', 'Error']].describe())

# 5. Check for "Imputation Artifacts"
# If 'BuildingArea' or 'YearBuilt' are exactly the median/mode values we used for imputation,
# it means we predicted based on guessed data.
# (We need to check the raw values, but let's look for patterns first)

print("\n🔍 Feature Analysis of Failures (Mean values vs Global Mean):")
# Compare the "Worst" rows vs the "Average" house in X_test
comparison = pd.DataFrame({
    'Worst_10_Mean': worst_combined.mean(numeric_only=True),
    'Global_Test_Mean': X_test.mean(numeric_only=True)
})
display(comparison.loc[['Rooms', 'Distance', 'Bathroom', 'Car', 'Landsize', 'BuildingArea']])

# 6. Visual Check
# Are we under-predicting or over-predicting?
under_estimated = (worst_combined['Error'] > 0).sum()
over_estimated = (worst_combined['Error'] < 0).sum()

print(f"\n📉 Diagnosis:")
print(f"   Under-Estimated (Real Price was Higher): {under_estimated}")
print(f"   Over-Estimated (Real Price was Lower):   {over_estimated}")

if under_estimated > over_estimated:
    print("   👉 CONCLUSION: The model tends to UNDER-value these outliers (likely Luxury homes).")
else:
    print("   👉 CONCLUSION: The model tends to OVER-value these outliers (likely Dump/Teardown homes).")